# [Going Deeper NLP 10] 더 멋진 번역기 만들기
transformer 모델을 이용해 한영 번역기를 만든다.

---

## 프로젝트 목표
---
- transformer 모델을 이해, 구현한다. 

## 프로젝트 설명
---
1. 데이터 다운로드 : [링크](https://github.com/jungyeul/korean-parallel-corpora/blob/master/korean-english-news-v1/korean-english-park.train.tar.gz)
2. 데이터 정제 
    - 중복, 결측치 제거
    - 한글 데이터 처리
    - 영어 소문자 변환
    - 토큰 추가 
3. 데이터 토큰화 
4. 모델 설계 
5. 훈련 및 평가 

    
## 0. Import module, library

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

print("한글 적용 완료")

한글 적용 완료


In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import re
import os
import io
import time
import random

import seaborn # Attention 시각화를 위해 필요!

print(tf.__version__)

2.1.0-rc2


## 1. 데이터 정제 및 토큰화
- 데이터 중복 제거 

In [5]:
data_dir = os.getenv('HOME')+'/aiffel/transformer/data'
kor_path = data_dir+"/korean-english-park.train.ko"
eng_path = data_dir+"/korean-english-park.train.en"

def clean_corpus(kor_path, eng_path):
    with open(kor_path, "r") as f: 
        kor = f.read().splitlines()
    with open(eng_path, "r") as f: 
        eng = f.read().splitlines()
    assert len(kor) == len(eng)

    raw = zip(kor, eng)
    cleaned_corpus = set(raw)

    return cleaned_corpus

cleaned_corpus = clean_corpus(kor_path, eng_path)

In [6]:
kor_corpus, eng_corpus = zip(*cleaned_corpus)
print(len(kor_corpus), len(eng_corpus))

78968 78968


- 데이터 정제
    1. 영문 소문자 변환
    2. 알파벳, 문장부호, 한글 제외는 제거
    3. 문장부호 양 옆에 공백 추가 
    4. 불필요한 공백 제거

In [7]:
def preprocess_sentence(sentence):
    
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!가-힣ㄱ-ㅎㅏ-ㅣ]+", " ", sentence)

    sentence = sentence.strip()
    
    return sentence

In [8]:
enc_corpus = []
dec_corpus = []

for kor, eng in zip(kor_corpus, eng_corpus):
    temp_kor = preprocess_sentence(kor)
    temp_eng = preprocess_sentence(eng)

    enc_corpus.append(temp_kor)
    dec_corpus.append(temp_eng)
    
print('korean data size:', len(enc_corpus))
print('english data size:', len(dec_corpus))
print("Korean:", enc_corpus[500])   
print("English:", dec_corpus[500])  

korean data size: 78968
english data size: 78968
Korean: 오바마 정권인수팀 부시 행정명령 전면 재검토 중
English: obama team reviewing virtually every agency aide says


- 데이터 토큰화
    - Sentencepiece활용 

# 루브릭 평가
---
1. **번역기 모델 학습에 필요한 텍스트 데이터 전처리가 잘 이루어졌다.**   
    - 데이터 정제, SentencePiece를 활용한 토큰화 및 데이터셋 구축의 과정이 지시대로 진행되었다.


2. **Transformer 번역기 모델이 정상적으로 구동된다.**

    - Transformer 모델의 학습과 추론 과정이 정상적으로 진행되어, 한-영 번역기능이 정상 동작한다.


3. **테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.** 
 
     - 제시된 문장에 대한 그럴듯한 영어 번역문이 생성되며, 시각화된 Attention Map으로 결과를 뒷받침한다.


    
# 회고
---
## 어려웠던 부분 
- 

## 알아낸 점 혹은 모호한 부분 
- 

## 느낀 점 
- 